In [21]:
# Ensure required packages are installed in the running kernel before importing them
# (using %pip ensures the package is installed into the kernel environment)
# Add 'pexpect' to avoid ModuleNotFoundError: No module named 'pexpect'
%pip install "pydantic>=2.0.0" python-dotenv semantic-kernel openai pexpect

import json
import os

from dotenv import load_dotenv

# pydantic may be reported as "could not be resolved" by some linters/LS if not
# installed in the editor environment; installing above for the kernel fixes runtime.
# Add a type-ignore to suppress static linter warnings in editors if needed.
from pydantic import BaseModel, ValidationError, Field  # type: ignore
from typing import List

# Add a type-ignore so static analysis in the editor won't flag the import when
# the package is installed at runtime by the magic %pip command above.
from openai import AsyncOpenAI  # type: ignore

# Import the Semantic Kernel OpenAI connector with fallbacks for different package layouts.
# Use importlib to attempt multiple candidate module paths and add type-ignore comments to
# suppress static analyzer warnings when packages are installed at runtime via %pip above.
from importlib import import_module

def _try_import_candidates(candidates, names):
	for mod_path in candidates:
		try:
			mod = import_module(mod_path)
			return tuple(getattr(mod, name) for name in names)
		except Exception:
			continue
	raise ImportError(f"Could not import {names} from any of: {candidates}")

connector_candidates = [
	"semantic_kernel.connectors.ai.open_ai",
	"semantic_kernel.connectors.ai.openai",
	"semantic_kernel.connectors.ai",
]

try:
	OpenAIChatCompletion, OpenAIChatPromptExecutionSettings = _try_import_candidates(
		connector_candidates,
		["OpenAIChatCompletion", "OpenAIChatPromptExecutionSettings"],
	)
except ImportError:
	# Provide a clearer error if imports fail at runtime (after %pip install above)
	raise

# Import agents with fallback paths to handle different package layouts across versions.
# Try the aggregated agents module first, then fall back to importing classes from submodules.
try:
	from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread  # type: ignore
except Exception:
	try:
		from semantic_kernel.agents.chat_completion_agent import ChatCompletionAgent  # type: ignore
		from semantic_kernel.agents.chat_history_agent_thread import ChatHistoryAgentThread  # type: ignore
	except Exception:
		# If these imports still fail, raise an informative error.
		raise ImportError(
			"Could not import ChatCompletionAgent and ChatHistoryAgentThread from semantic_kernel. "
			"Ensure 'semantic-kernel' is installed in the notebook kernel (use the %pip install line above)."
		)

# Import KernelArguments with fallbacks for different semantic-kernel package layouts.
try:
	KernelArguments = _try_import_candidates(
		[
			"semantic_kernel.functions",
			"semantic_kernel.kernel",
			"semantic_kernel.core",
		],
		["KernelArguments"],
	)[0]
except Exception:
	# Provide a clearer error if imports fail at runtime
	raise ImportError(
		"Could not import 'KernelArguments' from semantic_kernel. "
		"Ensure 'semantic-kernel' is installed in the notebook kernel (use the %pip install line above)."
	)

Note: you may need to restart the kernel to use updated packages.


In [22]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [23]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [24]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = OpenAIChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [25]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

You should see sample output similar to:

```json
User:
Create a travel plan for a family of 4, with 2 kids, from Singapore to Melboune
Validated Travel Plan:
{
    "main_task": "Plan a family trip from Singapore to Melbourne for 4 people including 2 kids.",
    "subtasks": [
        {
            "assigned_agent": "FlightBooking",
            "task_details": "Book round-trip flights from Singapore to Melbourne for 2 adults and 2 children."
        },
        {
            "assigned_agent": "HotelBooking",
            "task_details": "Find and book a family-friendly hotel in Melbourne that accommodates 4 people."
        },
        {
            "assigned_agent": "CarRental",
            "task_details": "Arrange for a car rental in Melbourne suitable for a family of 4."
        },
        {
            "assigned_agent": "ActivitiesBooking",
            "task_details": "Plan and book family-friendly activities in Melbourne suitable for kids."
        },
        {
            "assigned_agent": "DestinationInfo",
            "task_details": "Provide information about Melbourne, including attractions, dining options, and family-oriented activities."
        }
    ]
}
```